In [24]:
import nibabel as nib
import numpy as np
import glob,os
from scipy.stats import ttest_ind
import plotly.express as px

In [36]:
BIDS_DIR="/NAS/coolio/protocoles/CINOCIS/BIDS/"
DERIVATIVES_DIR=f"{BIDS_DIR}derivatives_v2.0/"

In [ ]:
# Liste des fichiers NIfTI à fusionner
file_list = glob.glob(r'/NAS/tupac/protocoles/PULSE/DTI/FA/*_onMNI.nii.gz')

# Initialiser une variable pour accumuler la somme des images
sum_image = None

# Lire chaque fichier et accumuler la somme
for file in file_list:
    img = nib.load(file)
    img_data = img.get_fdata()

    if sum_image is None:
        sum_image = np.zeros_like(img_data, dtype=np.float32)

    sum_image += img_data

# Calculer la moyenne
mean_image_data = sum_image / len(file_list)

# Créer une nouvelle image NIfTI pour la moyenne
mean_img = nib.Nifti1Image(mean_image_data, img.affine, img.header)

# Sauvegarder l'image moyenne
nib.save(mean_img, r'/NAS/tupac/protocoles/PULSE/DTI/FA/mean_image.nii.gz')


In [ ]:
# Liste des fichiers NIfTI à fusionner
file_list = glob.glob(r'/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/template_PAM50/sub-*/ses-*/sub-*_ses-M*_dwi_fa_on_PAM50.nii.gz')

# Initialiser une variable pour accumuler la somme des images
sum_image = None

# Lire chaque fichier et accumuler la somme
for file in file_list:
    img = nib.load(file)
    img_data = img.get_fdata()

    if sum_image is None:
        sum_image = np.zeros_like(img_data, dtype=np.float32)

    sum_image += img_data

# Calculer la moyenne
mean_image_data = sum_image / len(file_list)

# Créer une nouvelle image NIfTI pour la moyenne
mean_img = nib.Nifti1Image(mean_image_data, img.affine, img.header)

# Sauvegarder l'image moyenne
nib.save(mean_img, r'/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/mean_image_FA_onPAM50.nii.gz')

In [ ]:
tsv_path = BIDS_DIR + 'participants.tsv'
df = pd.read_csv(tsv_path, sep='\t')
df_M3M6 = pd.DataFrame(columns=[
    'Subject',
    'mean_fa_lesion_M3', 'mean_fa_nawm_M3', 'mean_adc_lesion_M3', 'mean_adc_nawm_M3',
    'mean_fa_lesion_M6', 'mean_fa_nawm_M6', 'mean_adc_lesion_M6', 'mean_adc_nawm_M6',
    'vol_lesion_M3', 'vol_nawm_M3', 'vol_lesion_M6', 'vol_nawm_M6', 'mean_fa_M3_M6_lesion', 'mean_adc_M3_M6_lesion'
])

for SUBJ in df['participant_id']:
    # print(SUBJ)
    lesion_M3_path = f"{DERIVATIVES_DIR}segmentation/{SUBJ}/ses-M3/lesion_ses-M3.lps.nii.gz"
    NAWM_M3_path = f"{DERIVATIVES_DIR}segmentation/{SUBJ}/ses-M3/NAWM_ses-M3.nii.gz"
    sc_M3_path = f"{DERIVATIVES_DIR}segmentation/{SUBJ}/ses-M3/mask_cord_normalised.nii.gz"
    reg_pattern = f"{DERIVATIVES_DIR}registration/{SUBJ}/ses-M3/{SUBJ}_ses-M3_acq-*dwi_fa.rec.nii.gz"
    fa_M3 = glob.glob(reg_pattern)
    adc_M3 = glob.glob(reg_pattern.replace('fa', 'adc'))
    
    lesion_M6_path = f"{DERIVATIVES_DIR}segmentation/{SUBJ}/ses-M6/lesion_ses-M6.lps.nii.gz"
    NAWM_M6_path = f"{DERIVATIVES_DIR}segmentation/{SUBJ}/ses-M6/NAWM_ses-M6.nii.gz"
    sc_M6_path = f"{DERIVATIVES_DIR}segmentation/{SUBJ}/ses-M6/mask_cord_normalised.nii.gz"
    reg_pattern = f"{DERIVATIVES_DIR}registration/{SUBJ}/ses-M6/{SUBJ}_ses-M6_acq-*dwi_fa.rec.nii.gz"
    fa_M6 = glob.glob(reg_pattern)
    adc_M6 = glob.glob(reg_pattern.replace('fa', 'adc'))

    # M3
    if fa_M3 and adc_M3 and os.path.exists(lesion_M3_path) and os.path.exists(NAWM_M3_path):
        fa_img_M3 = nib.load(fa_M3[0]).get_fdata()
        adc_img_M3 = nib.load(adc_M3[0]).get_fdata()
        lesion_mask_M3 = nib.load(lesion_M3_path).get_fdata()
        nawm_mask_M3 = nib.load(NAWM_M3_path).get_fdata()
        sc_M3 = nib.load(sc_M3_path).get_fdata()
        vol_lesion_M3 = np.sum(lesion_mask_M3 == 1)
        vol_nawm_M3 = np.sum(nawm_mask_M3 == 1)
        
        mean_fa_lesion_M3 = np.mean(fa_img_M3[(lesion_mask_M3 == 1) & (fa_img_M3 != 0) & (sc_M3 == 1)]) if vol_lesion_M3 > 0 else np.nan
        mean_fa_nawm_M3 = np.mean(fa_img_M3[(nawm_mask_M3 == 1) & (fa_img_M3 != 0) & (sc_M3 == 1)]) if vol_nawm_M3 > 0 else np.nan
        mean_adc_lesion_M3 = np.mean(adc_img_M3[(lesion_mask_M3 == 1) & (adc_img_M3 != 0) & (sc_M3 == 1)]) if vol_lesion_M3 > 0 else np.nan
        mean_adc_nawm_M3 = np.mean(adc_img_M3[(nawm_mask_M3 == 1) & (adc_img_M3 != 0) & (sc_M3 == 1)]) if vol_nawm_M3 > 0 else np.nan
    else:
        mean_fa_lesion_M3 = mean_fa_nawm_M3 = mean_adc_lesion_M3 = mean_adc_nawm_M3 = np.nan
        vol_lesion_M3 = vol_nawm_M3 = 0

    # M6
    if fa_M6 and adc_M6 and os.path.exists(lesion_M6_path) and os.path.exists(NAWM_M6_path):
        fa_img_M6 = nib.load(fa_M6[0]).get_fdata()
        adc_img_M6 = nib.load(adc_M6[0]).get_fdata()
        lesion_mask_M6 = nib.load(lesion_M6_path).get_fdata()
        nawm_mask_M6 = nib.load(NAWM_M6_path).get_fdata()
        sc_M6= nib.load(sc_M6_path).get_fdata()

        vol_lesion_M6 = np.sum(lesion_mask_M6 == 1)
        vol_nawm_M6 = np.sum(nawm_mask_M6 == 1)

        mean_fa_lesion_M6 = np.mean(fa_img_M6[(lesion_mask_M6 == 1) & (fa_img_M6 != 0) & (sc_M6 == 1)]) if vol_lesion_M6 > 0 else np.nan
        mean_fa_nawm_M6 = np.mean(fa_img_M6[(nawm_mask_M6 == 1) & (fa_img_M6 != 0) & (sc_M6 == 1)]) if vol_nawm_M6 > 0 else np.nan
        mean_adc_lesion_M6 = np.mean(adc_img_M6[(lesion_mask_M6 == 1) & (adc_img_M6 != 0) & (sc_M6 == 1)]) if vol_lesion_M6 > 0 else np.nan
        mean_adc_nawm_M6 = np.mean(adc_img_M6[(nawm_mask_M6 == 1) & (adc_img_M6 != 0) & (sc_M6 == 1)]) if vol_nawm_M6 > 0 else np.nan
    else:
        mean_fa_lesion_M6 = mean_fa_nawm_M6 = mean_adc_lesion_M6 = mean_adc_nawm_M6 = np.nan
        vol_lesion_M6 = vol_nawm_M6 = 0
    
    #Prediction
    adc_M3_M6_path=f"{DERIVATIVES_DIR}prediction_M3_M6/{SUBJ}/adc_M3_warpedByFlirt.nii.gz"
    fa_M3_M6_path=f"{DERIVATIVES_DIR}prediction_M3_M6/{SUBJ}/fa_M3_warpedByFlirt.nii.gz"
    lesion_M3onM6_path=f"{DERIVATIVES_DIR}prediction_M3_M6/{SUBJ}/lesion_dil_M3_warpedByFlirt.nii.gz"
    if os.path.exists(adc_M3_M6_path) and os.path.exists(fa_M3_M6_path) and os.path.exists(lesion_M3onM6_path):
        adc_M3_M6_img = nib.load(adc_M3_M6_path).get_fdata()
        fa_M3_M6_img = nib.load(fa_M3_M6_path).get_fdata()
        lesion_M3onM6_mask = nib.load(lesion_M3onM6_path).get_fdata()
        
        # Mask: voxels in lesion_mask_M6 but not in lesion_M3onM6_mask
        if lesion_mask_M6.shape != lesion_M3onM6_mask.shape:
            print(f"Shape mismatch for subject {SUBJ}:")
            print(f"lesion_mask_M6: {lesion_M6_path} shape={lesion_mask_M6.shape}")
            print(f"lesion_M3onM6_mask: {lesion_M3onM6_path} shape={lesion_M3onM6_mask.shape}")
            mean_adc_M3_M6_lesion = np.nan
            mean_fa_M3_M6_lesion = np.nan
        else:
            mask = (lesion_mask_M6 == 1) & (lesion_M3onM6_mask == 0) & (sc_M6 == 1)
            mean_adc_M3_M6_lesion = np.mean(adc_M3_M6_img[mask & (adc_M3_M6_img != 0)]) if np.sum(mask) > 0 else np.nan
            mean_fa_M3_M6_lesion = np.mean(fa_M3_M6_img[mask & (fa_M3_M6_img != 0)]) if np.sum(mask) > 0 else np.nan
    else:
        mean_adc_M3_M6_lesion = np.nan
        mean_fa_M3_M6_lesion = np.nan

    # Visualization command
    if len(adc_M6)>0 :
        if os.path.exists(adc_M6[0]) and os.path.exists(adc_M3_M6_path) and os.path.exists(lesion_M3onM6_path) and os.path.exists(NAWM_M6_path):
            print(f'freeview {adc_M6[0]}:grayscale=0.0,0.004 {adc_M3_M6_path}:grayscale=0.0,0.004 {lesion_M3onM6_path}:colormap=binary:binary_color=blue {DERIVATIVES_DIR}segmentation/{SUBJ}/ses-M6/lesion_ses-M6.lps.nii.gz:colormap=binary:binary_color=red {NAWM_M6_path}:colormap=binary:binary_color=green')


    df_M3M6.loc[len(df_M3M6)] = [
        SUBJ,
        mean_fa_lesion_M3, mean_fa_nawm_M3, mean_adc_lesion_M3, mean_adc_nawm_M3,
        mean_fa_lesion_M6, mean_fa_nawm_M6, mean_adc_lesion_M6, mean_adc_nawm_M6,
        vol_lesion_M3, vol_nawm_M3, vol_lesion_M6, vol_nawm_M6, mean_fa_M3_M6_lesion, mean_adc_M3_M6_lesion,
    ]


FileNotFoundError: No such file or no access: '/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/segmentation/sub-004/mask_cord_normalised.nii.gz'

In [63]:
df_M3M6

,Subject,mean_fa_lesion_M3,mean_fa_nawm_M3,mean_adc_lesion_M3,mean_adc_nawm_M3,mean_fa_lesion_M6,mean_fa_nawm_M6,mean_adc_lesion_M6,mean_adc_nawm_M6,vol_lesion_M3,vol_nawm_M3,vol_lesion_M6,vol_nawm_M6,mean_fa_M3_M6_lesion,mean_adc_M3_M6_lesion
0,sub-001,NaN,NaN,NaN,NaN,0.526794,0.558759,0.001078,0.001063,0,0,520,1638,NaN,NaN
1,sub-002,NaN,NaN,NaN,NaN,0.502577,0.556160,0.001049,0.001000,0,0,94,2014,NaN,NaN
2,sub-003,NaN,NaN,NaN,NaN,0.610819,0.501941,0.001121,0.001276,0,0,9,1523,NaN,NaN
3,sub-004,0.318732,0.606828,0.002103,0.001246,0.299763,0.689350,0.002036,0.000909,194,1349,135,1498,NaN,NaN
4,sub-005,0.438502,0.588982,0.001502,0.001074,0.180467,0.225567,0.002902,0.001992,1003,1783,1189,1827,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,sub-129,0.686681,0.617756,0.001299,0.001587,0.408295,0.531112,0.002102,0.001698,2,2174,15,1600,NaN,NaN
123,sub-130,0.313207,0.454163,0.002953,0.001989,0.409958,0.581876,0.002409,0.001363,691,2224,517,2078,NaN,NaN
124,sub-131,0.192393,0.506627,0.003814,0.001540,0.221902,0.525506,0.002938,0.001360,126,1675,120,1710,NaN,NaN
125,sub-132,0.394648,0.444914,0.001858,0.002021,0.455549,0.545895,0.001432,0.001373,567,1786,240,1977,NaN,NaN


In [66]:
# Graphique FA
fig_fa = px.box(
    df_M3M6.melt(id_vars=["Subject"], value_vars=["mean_fa_lesion_M3", "mean_fa_nawm_M3", "mean_fa_lesion_M6", "mean_fa_nawm_M6", "mean_fa_M3_M6_lesion"]),
    x="variable",
    y="value",
    color="variable",
    hover_data=["Subject"],
    title="Boxplots interactifs de FA_NAWM et FA_lesion"
)
for variable in ["mean_fa_lesion_M3", "mean_fa_nawm_M3", "mean_fa_lesion_M6", "mean_fa_nawm_M6", "mean_fa_M3_M6_lesion"]:
    fig_fa.add_scatter(
        x=[variable]*len(df_M3M6),
        y=df_M3M6[variable],
        mode='markers',
        marker=dict(size=6, opacity=0.6),
        name=f'{variable} points',
        text=df_M3M6['Subject'],
        hovertemplate='Subject=%{text}<br>FA=%{y}<extra></extra>',
        showlegend=False
    )
fig_fa.show()

# Graphique ADC
fig_adc = px.box(
    df_M3M6.melt(id_vars=["Subject"], value_vars=["mean_adc_lesion_M3", "mean_adc_nawm_M3", "mean_adc_lesion_M6", "mean_adc_nawm_M6", "mean_adc_M3_M6_lesion"]),
    x="variable",
    y="value",
    color="variable",
    hover_data=["Subject"],
    title="Boxplots interactifs de ADC_NAWM et ADC_lesion"
)
for variable in ["mean_adc_lesion_M3", "mean_adc_nawm_M3", "mean_adc_lesion_M6", "mean_adc_nawm_M6", "mean_adc_M3_M6_lesion"]:
    fig_adc.add_scatter(
        x=[variable]*len(df_M3M6),
        y=df_M3M6[variable],
        mode='markers',
        marker=dict(size=6, opacity=0.6),
        name=f'{variable} points',
        text=df_M3M6['Subject'],
        hovertemplate='Subject=%{text}<br>ADC=%{y}<extra></extra>',
        showlegend=False
    )
fig_adc.show()



In [72]:
t_stat, p_value = ttest_ind(df_M3M6["mean_fa_lesion_M3"].dropna(), df_M3M6["mean_fa_lesion_M6"].dropna(), equal_var=False)
print(f"difference M3 M6 fa lesion : ({round(t_stat,3)}, {round(p_value,3)})")

t_stat, p_value = ttest_ind(df_M3M6["mean_adc_lesion_M3"].dropna(), df_M3M6["mean_adc_lesion_M6"].dropna(), equal_var=False)
print(f"difference M3 M6 adc lesion : ({round(t_stat,3)}, {round(p_value,3)})")

t_stat, p_value = ttest_ind(df_M3M6["mean_fa_nawm_M3"].dropna(), df_M3M6["mean_fa_nawm_M6"].dropna(), equal_var=False)
print(f"difference M3 M6 fa nawm : ({round(t_stat,3)}, {round(p_value,3)})")

t_stat, p_value = ttest_ind(df_M3M6["mean_adc_nawm_M3"].dropna(), df_M3M6["mean_adc_nawm_M6"].dropna(), equal_var=False)
print(f"difference M3 M6 adc nawm : ({round(t_stat,3)}, {round(p_value,3)})")

t_stat, p_value = ttest_ind(
    pd.concat([df_M3M6["mean_fa_lesion_M3"].dropna(), df_M3M6["mean_fa_lesion_M6"].dropna()]),
    pd.concat([df_M3M6["mean_fa_nawm_M3"].dropna(), df_M3M6["mean_fa_nawm_M6"].dropna()]),
    equal_var=False
)
print(f"difference de fa entre lesion Vs NAWM : ({round(t_stat,3)}, {round(p_value,3)})")

t_stat, p_value = ttest_ind(
    pd.concat([df_M3M6["mean_adc_lesion_M3"].dropna(), df_M3M6["mean_adc_lesion_M6"].dropna()]),
    pd.concat([df_M3M6["mean_adc_nawm_M3"].dropna(), df_M3M6["mean_adc_nawm_M6"].dropna()]),
    equal_var=False
)
print(f"difference de adc entre lesion Vs NAWM : ({round(t_stat,3)}, {round(p_value,3)})")

t_stat, p_value = ttest_ind(
    pd.concat([df_M3M6["mean_fa_lesion_M3"].dropna(), df_M3M6["mean_fa_lesion_M6"].dropna()]),
    df_M3M6["mean_fa_M3_M6_lesion"].dropna(),
    equal_var=False
)
print(f"difference de fa pour prediction : ({round(t_stat,3)}, {round(p_value,3)})")

t_stat, p_value = ttest_ind(
    pd.concat([df_M3M6["mean_fa_lesion_M3"].dropna(), df_M3M6["mean_fa_lesion_M6"].dropna()]),
    df_M3M6["mean_fa_M3_M6_lesion"].dropna(),
    equal_var=False
)
print(f"difference de adc pour prediction : ({round(t_stat,3)}, {round(p_value,3)})")


difference M3 M6 fa lesion : (-0.359, 0.72)
difference M3 M6 adc lesion : (0.427, 0.67)
difference M3 M6 fa nawm : (-0.598, 0.55)
difference M3 M6 adc nawm : (0.693, 0.489)
difference de fa entre lesion Vs NAWM : (-7.163, 0.0)
difference de adc entre lesion Vs NAWM : (4.846, 0.0)
difference de fa pour prediction : (2.028, 0.045)
difference de adc pour prediction : (2.028, 0.045)


In [ ]:
# Effectuer un test t pour comparer les deux groupesv
t_stat, p_value = ttest_ind([df_M3M6["mean_fa_lesion_M3"].dropna(),df_M3M6["mean_fa_lesion_M6"].dropna()], fa_lesion_alues.dropna(), equal_var=False)

# Afficher les résultats du test t
print(f"T-statistic: {t_stat}, P-value: {p_value}")

In [ ]:
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
import os
%matplotlib inline
import matplotlib.pyplot as plt

# # Lire les fichiers avec nibabel
# #images = [ for file_path in file_paths]
# # Soustraire l'image '801T2W_SPAIR2D_T2w_masklesioncervicalcord_dilated' de 'seg_cerv_rpi_labeled_eroded'
# dilated_les_M6 = nib.load('/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/sct_4.3_fused2/sub-012/ses-M6/sub-012_ses-M6_acq-801T2W_SPAIR2D_T2w_masklesioncervicalcord_dilated.nii.gz').get_fdata()
# eroded_les_M6 = nib.load('/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/sct_4.3_fused2/sub-012/ses-M6/sub-012_ses-M6_acq-801T2W_SPAIR2D_T2w_masklesioncervicalcord_eroded.nii.gz').get_fdata()
# segmented_vert_M6 = nib.load('/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/sct_4.3_fused2/sub-012/ses-M6/seg_cerv_rpi_labeled_eroded.nii.gz').get_fdata()


# # Ne garder que les labels 3 à 6 dans dilated_img
# filtered_vert_data = np.where((segmented_vert_M6 >= 3) & (segmented_vert_M6 <= 6), 1, 0)
# result_vertminusles_data = filtered_vert_data - dilated_les_M6 - dilated_les_M3

# Chemin des fichiers à récupérer
#file_pattern = r'/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/*/fa_NAWM_mean.txt'
file_pattern = r'/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/*/fa_M3onM6.nii.gz'

# Récupérer les fichiers avec une taille supérieure à 0 octet
valid_files = [file for file in glob.glob(file_pattern) if os.path.getsize(file) > 0]

# Afficher les fichiers récupérés
print("Fichiers valides :", valid_files)
# Créer un dictionnaire pour stocker les données
data = {}


# Afficher des informations sur chaque fichier chargé
for i, img in enumerate(valid_files[:]):    
    SUBJ = img.split('/')[8]
    # print(f"Traitement du fichier ${SUBJ} {i+1}/{len(valid_files[:])}: {img}")
    if os.path.exists(img.replace('fa_M3onM6.nii.gz','fa_M3onM6.rotated.nii.gz')) :
        DWI_quant = nib.load(img.replace('fa_M3onM6.nii.gz','fa_M3onM6.rotated.nii.gz')).get_fdata()
        print(f"Vérification des fichiers : {img.replace('fa_M3onM6.nii.gz','fa_M3onM6.rotated.nii.gz')}")
    else :
        DWI_quant = nib.load(img).get_fdata()

    # Calculer la moyenne de DWI_quant pour les voxels dans filtered_vert_data
    nawm_name = f'/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/{SUBJ}/NAWM.nii.gz'
    lesion_name = f'/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/{SUBJ}/mask_lesion_M6_eroded_3_6.nii.gz'
    
    if os.path.exists(nawm_name) and os.path.exists(lesion_name):
        nawm_data = nib.load(nawm_name).get_fdata()
        mean_nawm = np.mean(DWI_quant[(nawm_data == 1) & (DWI_quant != 0)])
        
        filtered_vert_data = nib.load(lesion_name).get_fdata()    
        # Calculer la moyenne de DWI_quant pour les voxels dans result_vertminusles_data
        print(np.sum(filtered_vert_data))
        if np.sum(filtered_vert_data) == 0:
            mean_lesion = 0
        else:
            mean_lesion = np.mean(DWI_quant[(filtered_vert_data == 1) & (DWI_quant != 0)])

        # Ajouter les résultats au dictionnaire
        data[SUBJ] = {'mean_nawm': mean_nawm, 'mean_lesion': mean_lesion}

# Créer un DataFrame à partir du dictionnaire
df_results = pd.DataFrame.from_dict(data, orient='index')

# Afficher le DataFrame
print(df_results)

# # Afficher l'histogramme
# plt.hist(DWI_quant[filtered_vert_data == 1], bins=50, color='blue', alpha=0.7)
# plt.title("Histogramme des valeurs de DWI_quant pour NAWM")
# plt.xlabel("Valeurs de DWI_quant")
# plt.ylabel("Fréquence")
# plt.show()    

Fichiers valides : ['/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-078/fa_M3onM6.nii.gz', '/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-126/fa_M3onM6.nii.gz', '/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-107/fa_M3onM6.nii.gz', '/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-048/fa_M3onM6.nii.gz', '/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-010/fa_M3onM6.nii.gz', '/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-040/fa_M3onM6.nii.gz', '/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-013/fa_M3onM6.nii.gz', '/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-090/fa_M3onM6.nii.gz', '/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-008/fa_M3onM6.nii.gz', '/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-098/fa_M3onM6.nii.gz', '/NAS/coolio/protocoles/CINOCIS/BIDS/der

/home/global/Anaconda3-2023.09/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning:

Mean of empty slice.

/home/global/Anaconda3-2023.09/lib/python3.11/site-packages/numpy/core/_methods.py:129: RuntimeWarning:

invalid value encountered in scalar divide



0.0
Vérification des fichiers : /NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-126/fa_M3onM6.rotated.nii.gz
0.0
0.0
Vérification des fichiers : /NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-048/fa_M3onM6.rotated.nii.gz
Vérification des fichiers : /NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-010/fa_M3onM6.rotated.nii.gz
251.0
Vérification des fichiers : /NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-040/fa_M3onM6.rotated.nii.gz
140.0
Vérification des fichiers : /NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-013/fa_M3onM6.rotated.nii.gz
153.0
Vérification des fichiers : /NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-090/fa_M3onM6.rotated.nii.gz
0.0
Vérification des fichiers : /NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-008/fa_M3onM6.rotated.nii.gz
70.0
Vérification des fichiers : /NAS/coolio/protocoles/CINOCIS/BIDS/derivati

In [ ]:
from scipy.stats import ttest_ind

import plotly.express as px

# Créer un DataFrame avec les données triées
# df_results = df_results.sort_values(by="FA_NAWM")

# Créer un graphique interactif avec Plotly
fig = px.box(
    df_results.melt(id_vars=["Subject"], value_vars=["mean_nawm", "mean_lesion"]),
    x="variable",
    y="value",
    color="variable",
    hover_data=["Subject"],
    labels={"value": "FA Values", "variable": "Group"},
    title="Boxplots interactifs de FA_NAWM et FA_lesion"
)

# Afficher le graphique
fig.show()

# # Effectuer un test t pour comparer les deux groupes
# t_stat, p_value = ttest_ind(fa_nawm_values.dropna(), fa_lesion_values.dropna(), equal_var=False)

# # Afficher les résultats du test t
# print(f"T-statistic: {t_stat}, P-value: {p_value}")

KeyError: "The following 'id_vars' are not present in the DataFrame: ['Subject']"

In [40]:
import glob
import os
import pandas as pd

# Chemin des fichiers à récupérer
file_pattern = r'/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/*/fa_NAWM_mean.txt'

# Récupérer les fichiers avec une taille supérieure à 0 octet
valid_files = [file for file in glob.glob(file_pattern) if os.path.getsize(file) > 0]

# Afficher les fichiers récupérés
print("Fichiers valides :", valid_files)
# Créer un dictionnaire pour stocker les données
data = {}

# Parcourir les fichiers valides
for file in valid_files:
    # Extraire la clé "sub-..." du chemin
    key = os.path.basename(os.path.dirname(file))
    
    # Lire la valeur FA_NAWM contenue dans le fichier
    value = float(open(file, 'r').read().strip())
    
    # Ajouter la clé et la valeur au dictionnaire
    #dta[key] = value

    # Chemin du fichier fa_lesion correspondant
    lesion_file = os.path.join(os.path.dirname(file), 'fa_lesion_M6_mean.txt')
    
    # Vérifier si le fichier fa_lesion existe et a une taille > 0
    if os.path.exists(lesion_file) and os.path.getsize(lesion_file) > 0:
        lesion_value = float(open(lesion_file, 'r').read().strip())
    else:
        lesion_value = 0  # Valeur par défaut si le fichier n'existe pas ou est vide
    
    # Ajouter la valeur fa_lesion au dictionnaire
    if value > 0 and lesion_value > 0:
        data[key] = {'FA_NAWM': value, 'FA_lesion': lesion_value}


# Créer un DataFrame à partir du dictionnaire
df_FA_NAWM = pd.DataFrame.from_dict(data, orient='index').reset_index()
df_FA_NAWM.rename(columns={'index': 'Subject'}, inplace=True)

# Afficher le DataFrame
print(df_FA_NAWM)

Fichiers valides : ['/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-078/fa_NAWM_mean.txt', '/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-126/fa_NAWM_mean.txt', '/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-107/fa_NAWM_mean.txt', '/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-010/fa_NAWM_mean.txt', '/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-040/fa_NAWM_mean.txt', '/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-013/fa_NAWM_mean.txt', '/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-090/fa_NAWM_mean.txt', '/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-008/fa_NAWM_mean.txt', '/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-098/fa_NAWM_mean.txt', '/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-111/fa_NAWM_mean.txt', '/NAS/coolio/protocoles/CINOCIS/BIDS/der

In [56]:
import nibabel as nib
import os
import pandas as pd
import glob
import numpy as np

# Initialiser une liste pour stocker les résultats
results = []
fa_files = glob.glob('/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-*/fa_M3onM6.nii.gz')
fa_files = sorted(fa_files)

# Parcourir tous les sujets possibles
for fa_path in fa_files:
    subj = os.path.basename(os.path.dirname(fa_path))
    # Pour chaque type de fichier (fa, adc)
    for modality in ['fa', 'adc']:
        modality_path = f'/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/{subj}/{modality}_M3onM6.lps.nii.gz'
        if os.path.exists(modality_path):
            nawm_path = f'/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/{subj}/NAWM_lps.nii.gz'
            mask_lesion_path = f'/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/{subj}/mask_lesion_M6_eroded_3_6_lps.nii.gz'
            print(f'patient {subj} - {modality}')
            if os.path.exists(nawm_path) and os.path.exists(mask_lesion_path):
                nawm_img = nib.load(nawm_path)
                mask_lesion_img = nib.load(mask_lesion_path)
                modality_img = nib.load(modality_path)

                nawm_data = nawm_img.get_fdata()
                mask_lesion_data = mask_lesion_img.get_fdata()
                modality_data = modality_img.get_fdata()

                mean_nawm = np.nanmean(modality_data[(nawm_data == 1) & (modality_data != 0)])
                mean_lesion = np.nanmean(modality_data[(mask_lesion_data == 1) & (modality_data != 0)])
                print(f'Mean {modality.upper()}_NAWM for {subj}: {mean_nawm}')
                print(f'Mean {modality.upper()}_lesion for {subj}: {mean_lesion}')
                results.append({'Subject': subj, f'{modality.upper()}_NAWM': mean_nawm, f'{modality.upper()}_lesion': mean_lesion})
                
# Créer un DataFrame à partir des résultats
df_results = pd.DataFrame(results)

# Afficher le DataFrame
print(df_results)


patient sub-003 - fa
patient sub-003 - adc
patient sub-004 - fa
Mean FA_NAWM for sub-004: 0.5697818013995174
Mean FA_lesion for sub-004: 0.6115499159746003
patient sub-004 - adc
Mean ADC_NAWM for sub-004: 0.0012156045936207204
Mean ADC_lesion for sub-004: 0.0009780332401166099
patient sub-005 - fa
Mean FA_NAWM for sub-005: 0.37307082658323565
Mean FA_lesion for sub-005: 0.4622087193501962
patient sub-005 - adc
Mean ADC_NAWM for sub-005: 0.0017567409724749713
Mean ADC_lesion for sub-005: 0.001218340901746037
patient sub-006 - fa
Mean FA_NAWM for sub-006: 0.43876212437468676
Mean FA_lesion for sub-006: 0.45259168381413456
patient sub-006 - adc
Mean ADC_NAWM for sub-006: 0.0015529629847512018
Mean ADC_lesion for sub-006: 0.0010216941003116074
patient sub-007 - fa
patient sub-007 - adc
patient sub-008 - fa
Mean FA_NAWM for sub-008: 0.398540123981588
Mean FA_lesion for sub-008: 0.3992381704705102
patient sub-008 - adc
Mean ADC_NAWM for sub-008: 0.0014434679080937605
Mean ADC_lesion for sub-

/tmp/ipykernel_12277/4150433355.py:32: RuntimeWarning:

Mean of empty slice



Mean FA_NAWM for sub-009: 0.4305846143468445
Mean FA_lesion for sub-009: nan
patient sub-009 - adc
Mean ADC_NAWM for sub-009: 0.0015198876478733465
Mean ADC_lesion for sub-009: nan
patient sub-010 - fa
Mean FA_NAWM for sub-010: 0.35724487081988826
Mean FA_lesion for sub-010: 0.33202400566097273
patient sub-010 - adc
Mean ADC_NAWM for sub-010: 0.0019372648449516242
Mean ADC_lesion for sub-010: 0.0015668623755589514
patient sub-011 - fa
Mean FA_NAWM for sub-011: nan
Mean FA_lesion for sub-011: nan
patient sub-011 - adc


/tmp/ipykernel_12277/4150433355.py:31: RuntimeWarning:

Mean of empty slice



Mean ADC_NAWM for sub-011: 0.0016095333330173262
Mean ADC_lesion for sub-011: 0.0014555944196347678
patient sub-012 - fa
Mean FA_NAWM for sub-012: 0.6224320192032108
Mean FA_lesion for sub-012: 0.6981053757217696
patient sub-012 - adc
Mean ADC_NAWM for sub-012: 0.001021016579501792
Mean ADC_lesion for sub-012: 0.0007946891051207511
patient sub-013 - fa
Mean FA_NAWM for sub-013: 0.26110341762741135
Mean FA_lesion for sub-013: 0.1508745139623
patient sub-013 - adc
Mean ADC_NAWM for sub-013: 0.0024151773299898563
Mean ADC_lesion for sub-013: 0.0028946413278117095
patient sub-014 - fa
Mean FA_NAWM for sub-014: 0.48170071081327187
Mean FA_lesion for sub-014: 0.6078253674010435
patient sub-014 - adc
Mean ADC_NAWM for sub-014: 0.0011225376418328645
Mean ADC_lesion for sub-014: 0.0010667241270615098
patient sub-015 - fa
Mean FA_NAWM for sub-015: 0.4269647807489128
Mean FA_lesion for sub-015: nan
patient sub-015 - adc
Mean ADC_NAWM for sub-015: 0.0015806987090841939
Mean ADC_lesion for sub-015:

In [54]:
fa_files = glob.glob('/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-*/fa_M3onM6.nii.gz')
fa_files = sorted(fa_files)

# Parcourir tous les sujets possibles
for fa_path in fa_files:
    subj = os.path.basename(os.path.dirname(fa_path))
    # Pour chaque type de fichier (fa, adc)
    for modality in ['fa', 'adc']:
        if os.path.exists(f'/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/{subj}/{modality}_M3onM6.rotated.nii.gz'):
            modality_path = f'/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/{subj}/{modality}_M3onM6.rotated.nii.gz'
            print(f'mri_convert {modality_path} --out_orientation LPS {modality_path.replace(".rotated.nii.gz", ".lps.nii.gz")}  ')
        elif os.path.exists(f'/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/{subj}/{modality}_M3onM6.nii.gz'):
            modality_path = f'/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/{subj}/{modality}_M3onM6.nii.gz'
            print(f'mri_convert {modality_path} --out_orientation LPS {modality_path.replace(".nii.gz", ".lps.nii.gz")}  ')
        else:
            print(f'Fichier introuvable pour {subj} et {modality}')


mri_convert /NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-003/fa_M3onM6.rotated.nii.gz --out_orientation LPS /NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-003/fa_M3onM6.lps.nii.gz  
mri_convert /NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-003/adc_M3onM6.rotated.nii.gz --out_orientation LPS /NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-003/adc_M3onM6.lps.nii.gz  
mri_convert /NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-004/fa_M3onM6.nii.gz --out_orientation LPS /NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-004/fa_M3onM6.lps.nii.gz  
mri_convert /NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-004/adc_M3onM6.nii.gz --out_orientation LPS /NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-004/adc_M3onM6.lps.nii.gz  
mri_convert /NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-005/fa_M3o

In [61]:
fa_files = glob.glob('/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-*/fa_M3onM6.nii.gz')
fa_files = sorted(fa_files)

# Parcourir tous les sujets possibles
for fa_path in fa_files:
    subj = os.path.basename(os.path.dirname(fa_path))
    if os.path.exists(f'/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/{subj}/fa_M3onM6.lps.nii.gz') and os.path.exists(f'/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/{subj}/adc_M3onM6.lps.nii.gz') and os.path.exists(f'/NAS/coolio/protocoles/CINOCIS/BIDS/{subj}/ses-M3/anat/recM3onM6Warped.nii.gz') and os.path.exists(f'/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/{subj}/NAWM_lps.nii.gz') and os.path.exists(f'/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/{subj}/mask_lesion_M6_eroded_3_6_lps.nii.gz'):
        print(f'freeview -v /NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/{subj}/fa_M3onM6.lps.nii.gz:grayscale=0.2,0.7 -v /NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/{subj}/adc_M3onM6.lps.nii.gz:grayscale=0.001,0.005 -v /NAS/coolio/protocoles/CINOCIS/BIDS/{subj}/ses-M3/anat/recM3onM6Warped.nii.gz -v /NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/{subj}/NAWM_lps.nii.gz:colormap=lut   /NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/{subj}/mask_lesion_M6_eroded_3_6_lps.nii.gz:colormap=lut:lut=ReducedLabels4')
    

freeview -v /NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-004/fa_M3onM6.lps.nii.gz:grayscale=0.2,0.7 -v /NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-004/adc_M3onM6.lps.nii.gz:grayscale=0.001,0.005 -v /NAS/coolio/protocoles/CINOCIS/BIDS/sub-004/ses-M3/anat/recM3onM6Warped.nii.gz -v /NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-004/NAWM_lps.nii.gz:colormap=lut   /NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-004/mask_lesion_M6_eroded_3_6_lps.nii.gz:colormap=lut:lut=ReducedLabels4
freeview -v /NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-005/fa_M3onM6.lps.nii.gz:grayscale=0.2,0.7 -v /NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-005/adc_M3onM6.lps.nii.gz:grayscale=0.001,0.005 -v /NAS/coolio/protocoles/CINOCIS/BIDS/sub-005/ses-M3/anat/recM3onM6Warped.nii.gz -v /NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-005/NAWM_lps.nii.gz:

In [80]:
fa_files = glob.glob('/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/*/fa_M3onM6.nii.gz')
fa_files = sorted(fa_files)

# Parcourir tous les sujets possibles
for fa_path in fa_files:
    subj = os.path.basename(os.path.dirname(fa_path))
    
    files_dir1 = glob.glob(f'/NAS/coolio/protocoles/CINOCIS/BIDS/{subj}/ses-M6/anat/*SPAIR2D_T2w*.nii.gz')
    fa_files = glob.glob(f'/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/{subj}/fa*.nii.gz')
    adc_file = glob.glob(f'/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/{subj}/adc*.nii.gz')
    print(f"freeview {files_dir1[0]} {fa_files[0]} {adc_file[0]}")

freeview /NAS/coolio/protocoles/CINOCIS/BIDS/sub-003/ses-M6/anat/sub-003_ses-M6_acq-1901T2W_SPAIR2D_T2w.nii.gz /NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-003/fa_M3onM6.nii.gz /NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-003/adc_M3onM6.nii.gz
freeview /NAS/coolio/protocoles/CINOCIS/BIDS/sub-004/ses-M6/anat/sub-004_ses-M6_acq-801T2W_SPAIR2D_T2w.nii.gz /NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-004/fa_M3onM6.nii.gz /NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-004/adc_M3onM6.nii.gz
freeview /NAS/coolio/protocoles/CINOCIS/BIDS/sub-005/ses-M6/anat/sub-005_ses-M6_acq-801T2W_SPAIR2D_T2w.nii.gz /NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-005/fa_M3onM6.nii.gz /NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-005/adc_M3onM6.nii.gz
freeview /NAS/coolio/protocoles/CINOCIS/BIDS/sub-006/ses-M6/anat/sub-006_ses-M6_acq-1801T2W_SPAIR2D_T2w.nii.gz /NAS/cool

In [ ]:
import glob

SUBJECT = 'sub-091'
print(df_results[df_results['Subject'] == SUBJECT])


# Afficher les fichiers du premier répertoire
files_dir1 = glob.glob(f'/NAS/coolio/protocoles/CINOCIS/BIDS/{SUBJECT}/ses-M6/anat/*SPAIR2D_T2w*.nii.gz')
for file in files_dir1:
    print(file)


# Afficher les fichiers du premier répertoire
files_dir1 = glob.glob(f'/NAS/coolio/protocoles/CINOCIS/BIDS/{SUBJECT}/ses-M3/anat/*SPAIR2D_T2w*.nii.gz')
for file in files_dir1:
    print(file)

# Afficher les fichiers du deuxième répertoire
files_dir2 = glob.glob(f'/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/{SUBJECT}/*.nii.gz')
print("Fichiers dans le deuxième répertoire:")
for file in files_dir2:
    print(file)


    Subject  FA_NAWM  FA_lesion
64  sub-091  0.40518    0.67741
Fichiers dans le premier répertoire:
/NAS/coolio/protocoles/CINOCIS/BIDS/sub-091/ses-M6/anat/sub-091_ses-M6_acq-1501T2W_SPAIR2D_T2w.nii.gz
Fichiers dans le premier répertoire:
/NAS/coolio/protocoles/CINOCIS/BIDS/sub-091/ses-M3/anat/sub-091_ses-M3_acq-1501T2W_SPAIR2D_T2w_res_RPI_seg.nii.gz
/NAS/coolio/protocoles/CINOCIS/BIDS/sub-091/ses-M3/anat/sub-091_ses-M3_acq-1501T2W_SPAIR2D_T2w.nii.gz
/NAS/coolio/protocoles/CINOCIS/BIDS/sub-091/ses-M3/anat/sub-091_ses-M3_acq-1501T2W_SPAIR2D_T2w_lesionseg_recM3onM6_warped.nii.gz
/NAS/coolio/protocoles/CINOCIS/BIDS/sub-091/ses-M3/anat/sub-091_ses-M3_acq-1501T2W_SPAIR2D_T2w_RPI_seg.nii.gz
Fichiers dans le deuxième répertoire:
/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-091/fa_M3onM6.nii.gz
/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-091/lesion_M3_dilated.nii.gz
/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-091/ad_

In [36]:
df_results.columns

Index(['Subject', 'FA_NAWM', 'FA_lesion', 'AD_NAWM', 'AD_lesion', 'ADC_NAWM',
       'ADC_lesion'],
      dtype='object')

In [57]:
from scipy.stats import ttest_ind

import plotly.express as px

# Créer un DataFrame avec les données triées
# df_FA_NAWM_sorted = df_results.sort_values(by="FA_NAWM")

# Créer un graphique interactif avec Plotly
fig = px.box(
    df_results.melt(id_vars=["Subject"], value_vars=["FA_NAWM", "FA_lesion"]),
    x="variable",
    y="value",
    color="variable",
    hover_data=["Subject"],
    labels={"value": "FA Values", "variable": "Group"},
    title="Boxplots interactifs de FA_NAWM et FA_lesion"
)

# Afficher le graphique
fig.show()

# Effectuer un test t pour comparer les deux groupes
t_stat, p_value = ttest_ind(df_results["FA_NAWM"].dropna(), df_results["FA_lesion"].dropna(), equal_var=False)

# Afficher les résultats du test t
print(f"T-statistic: {t_stat}, P-value: {p_value}")

T-statistic: -1.805705575835405, P-value: 0.07563556727257681


In [42]:
df_FA_NAWM_sorted_nolesionna = df_FA_NAWM_sorted.dropna(subset=['FA_lesion'])
lowest_fa_lesion_subjects = df_FA_NAWM_sorted_nolesionna.nsmallest(5, 'FA_lesion')
print(lowest_fa_lesion_subjects[['Subject', 'FA_lesion']])

    Subject  FA_lesion
64  sub-091   0.021713
70  sub-099   0.149393
8   sub-013   0.174345
3   sub-008   0.220028
5   sub-010   0.277457
